In [ ]:
import os
!pip install efficientnet
import efficientnet.keras as enet

save_directory= "Model"
train_address = "Train"
test_address = "Test"

if not os.path.exists(test_address):
  os.makedirs(test_address)
  !wget -q https://raw.githubusercontent.com/circulosmeos/gdown.pl/master/gdown.pl -O gdown.pl
  !pip install efficientnet
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=n51E&id=1rxlpshNKP2gQ6NP9Wpvq-jUdbk5t6-Ub" "Red-Cards.zip"
  !unzip Red-Cards.zip
  !mv Red-Cards Test/Red-Cards
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=Ez4P&id=1tvWzG3stQaY6fqs8O6F_3Ec_qguZU3mC" "Yellow-Cards.zip"
  !unzip Yellow-Cards.zip
  !mv Yellow-Cards Test/Yellow-Cards

if not os.path.exists(train_address):
  os.makedirs(train_address)
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=nJ31&id=1sQuc6JoE5CoKq6JYe6i75AoWEBngAVb8" "Red-Cards.zip"
  !unzip Red-Cards.zip
  !mv Red-Cards Train/Red-Cards
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=vrjS&id=1IozEkL1xhRoG9oqnBvdozrlTcTsqk9s6" "Yellow-Cards.zip"
  !unzip Yellow-Cards.zip
  !mv Yellow-Cards Train/Yellow-Cards


In [ ]:

import keras
from keras.callbacks import EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Multiply, add
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.regularizers import l2
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.utils.generic_utils import get_custom_objects
from keras.backend import sigmoid

import matplotlib.image as mpimg
import numpy as np
import keras.backend as K
import pathlib
from PIL import Image
import cv2
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import pandas as pd
import os
from keras.optimizers import Adam,RMSprop, SGD , Nadam, Adadelta
BATCH_SIZE = 16
test_nb = 1000
train_nb = 11000
num_classes = 2
img_size= 224
classes = []

train_path = "Train"
test_path = "Test"


class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))


get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   zoom_range=[1.0,2.0],
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=BATCH_SIZE,
        seed = 13)

validation_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        batch_size=BATCH_SIZE,
        seed = 13)
#%% finetuning resnet50

input_tensor = Input(shape=(img_size, img_size, 3))
model = enet.EfficientNetB0(include_top=False,input_tensor=input_tensor, pooling='avg', weights='imagenet')
x = model.output

x = BatchNormalization(axis=-1)(x)
x = Dropout(0.7)(x)

x = Dense(512)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation(swish_act)(x)
x = Dropout(0.5)(x)

x = Dense(512)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation(swish_act)(x)
x = Dense(2)(x)
categoryBranch = Activation("softmax", name="category_output")(x)

model = Model(
  inputs=input_tensor,
  outputs=categoryBranch,
  name="soccerNet")

opt = Adam(lr=0.001,  decay=0.0001)

    
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit_generator(train_generator,
                    steps_per_epoch=train_nb/ 16 ,
                    epochs=60,
                    validation_data=validation_generator,
                    validation_steps=int(test_nb/16),
                    verbose=1)  #, checkpointer

import datetime
now = datetime.datetime.now()
model.evaluate(validation_generator)


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("'model accuracy")
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("'model loss")

print("[INFO] serializing network...")
model.save("model_saved")

